In [1]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select

In [2]:
s=Service("/mnt/c/Users/silvi/Desktop/ironhack/proyectos/proyecto2/pipeline_proyecto1-2/jupyter/chromedriver.exe")
driver = webdriver.Chrome(service=s)
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)

opciones.headless=False    # si True, no aperece la ventana (headless=no visible)

opciones.add_argument('--start-maximized')         # comienza maximizado

opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies

#opciones.add_extension('driver_folder/adblock.crx')       # adblocker

opciones.add_argument('--incognito')              # incognito


In [3]:
url = ('https://www2.hm.com/es_es/index.html')
driver.get(url)
driver.implicitly_wait(2)

driver.find_element_by_css_selector('#onetrust-accept-btn-handler').click()

<ipython-input-3-b585bda75737>:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('#onetrust-accept-btn-handler').click()


In [4]:
body = driver.execute_script("return document.body")
source = body.get_attribute('innerHTML') 
soup_sc = BeautifulSoup(source, "html.parser")


In [5]:
enlace=soup_sc.findAll("a",{"class": "menu__sub-link"})
lista_enlaces = [f"https://www2.hm.com{elemento.get('href')}" for elemento in enlace]


In [6]:
novedades=[]
for i in lista_enlaces:
    if "mujer" in i or "hombre" in i :
        if "novedades" in i and "ver-todo" in i:
            url = (f'{i}')
            novedades.append(url)
    elif "novedades" in i:
        url = (f'{i}')
        novedades.append(url)
novedades
    

['https://www2.hm.com/es_es/mujer/novedades/ver-todo.html',
 'https://www2.hm.com/es_es/hombre/novedades/ver-todo.html',
 'https://www2.hm.com/es_es/bebe/recien-nacido/novedades.html',
 'https://www2.hm.com/es_es/bebe/nina/novedades.html',
 'https://www2.hm.com/es_es/bebe/ninos/novedades.html',
 'https://www2.hm.com/es_es/bebe/productos/novedades.html',
 'https://www2.hm.com/es_es/ninos/nina/novedades.html',
 'https://www2.hm.com/es_es/ninos/ninos/novedades.html',
 'https://www2.hm.com/es_es/home/novedades/nuevos-productos.html']

In [89]:
html=[]
for i in novedades: 
    url = f"{i}"
    driver.get(url)
    driver.implicitly_wait(2)
    
    try :
        for i in range(20):
            driver.find_element_by_css_selector('#page-content > div > div:nth-child(3) > div.load-more-products > button').click()
            body = driver.execute_script("return document.body")
            source = body.get_attribute('innerHTML') 
        html.append(source)
    except:
        body = driver.execute_script("return document.body")
        source = body.get_attribute('innerHTML') 
        html.append(source)
        

<ipython-input-89-0fb8ac2093d2>:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('#page-content > div > div:nth-child(3) > div.load-more-products > button').click()


In [8]:
# driver.find_element_by_css_selector('#menu-links > li:nth-child(1) > ul > li:nth-child(1)').click()

In [90]:
tabs=["mujer","hombre","recien_nacido","bebe_nina","bebe_nino","bebe_productos","nina","nino","home"]

In [91]:
dic={}
for x in range(9):
    dic[tabs[x]]=BeautifulSoup(html[x], "html.parser")
    


In [92]:
dic.keys()

dict_keys(['mujer', 'hombre', 'recien_nacido', 'bebe_nina', 'bebe_nino', 'bebe_productos', 'nina', 'nino', 'home'])

In [ ]:
#dic     color=dic[tabs[i]].findAll("a",{"class": "swatch"})
    dic2["colors_"+f"{tabs[i]}"]=[co.getText() for co in color]

In [93]:
dic2={}
for i in range(9):
    upc=dic[tabs[i]].findAll("h3",{"class": "item-heading"})
    numero=re.findall(r"\d{2,}", str(upc))
    dic2["prod_id_"+f"{tabs[i]}"]=numero
    
    productos=dic[tabs[i]].findAll("h3",{"class": "item-heading"})
    dic2["productos_"+f"{tabs[i]}"] = [producto.getText() for producto in productos]
    
    precio=dic[tabs[i]].findAll("span",{"class": "price regular"})
    dic2["precios_"+f"{tabs[i]}"]=[precios.getText() for precios in precio]




In [94]:
lista_1=list(dic2.keys())
lista_2=list(dic2.values())

In [95]:
mujer={lista_1[i]: lista_2[i] for i in range(3)}
df_m=pd.DataFrame(mujer)
df_m

ValueError: All arrays must be of the same length

In [85]:
mujer={lista_1[i]: lista_2[i] for i in range(3)}
hombre={lista_1[i]: lista_2[i] for i in range(4,8)}
recien_nacido={lista_1[i]: lista_2[i] for i in range(8,12)}
bebe_nina={lista_1[i]: lista_2[i] for i in range(12,16)}
bebe_nino={lista_1[i]: lista_2[i] for i in range(16,20)}
bebe_productos={lista_1[i]: lista_2[i] for i in range(20,24)}
nina={lista_1[i]: lista_2[i] for i in range(24,28)}
nino={lista_1[i]: lista_2[i] for i in range(28,32)}
home={lista_1[i]: lista_2[i] for i in range(32,36)}
home.keys()

IndexError: list index out of range

In [80]:

df = pd.DataFrame([[key, mujer[key]] for key in  mujer.keys()], columns=['prod_id', 'product','precios','color'])

ValueError: All arrays must be of the same length